# Notes on Pauli projectors

A Pauli projector on a **single qubit** can be defined as:


$$A = \frac{1}{2}\big( I \pm \sigma \big)$$

where $\sigma \in \{X,Y,Z\}$. The $+$ term above projects onto the $+1$ eigenvector and $-$ projects onto the $-1$ eigenvector

Next consider projection of an  $n$-qubits (requires taking products of single qubit projectors)

$$B = \prod_{i=0}^{n-1} \frac{1}{2}\Bigg[ \bigg(\bigotimes_{j=0}^{n-1} I \bigg) \pm \bigg( \bigotimes_{j=0}^{i-1} I \otimes \sigma_{i} \otimes \bigotimes_{j=i+1}^{n-1} I \bigg) \Bigg]$$

$$= \prod_{i=0}^{n-1} \Bigg( \frac{1}{2} \big( II \text{...}I_{i}\text{...}I  \pm II\text{...} \sigma_{i} \text{...} I\big) \Bigg)$$

- first term is all identity string
- second term is all identities bar one unique single Pauli matrix


In the computational basis $\sigma = Z$ and we get:


$$C = \prod_{i=0}^{n-1} \Bigg( \frac{1}{2} \big( II \text{...} I  \pm II\text{...} Z_{i} \text{...} I\big) \Bigg)$$

This product result in a binary expansion with all combinations of $Z$ on each qubit:


$$C = \frac{1}{2^{n}} \sum_{i=0}^{2^{n}-1} \pm P$$

Which bitstring is chosen to project onto determines signs in the expansion...


E.g. projection onto the state: $|10 \rangle = |2 \rangle$

$$|2 \rangle \langle 2| = \frac{1}{2^{2}}\big( II - ZI \big) \big( II + IZ \big)$$
$$ = \frac{1}{4} \big( II + IZ - ZI - ZZ \big)$$

Let the set $\mathcal{W}$ be all $2^{n}$ the binary bitstrings of size $n$, then we have: for projecting onto a particular computational basis state $| b \rangle$:

$$|b \rangle \langle b| = \frac{1}{2^{n}} \sum_{k \in  \mathcal{W}}\Bigg[ (-1)^{(b\cdot k) \: mod \: 2}\bigg( \bigotimes_{i=0}^{n-1} \sigma_{i}^{k[i]}  \bigg) \Bigg]$$

where:
- $k[i]$ is the $i$-th bit of the bitstring $| k \rangle$ this determines each $\sigma \in \{I, Z \}$:
    - $\sigma_{i}^{0} = I_{i}$
    - $\sigma_{i}^{1} = Z_{i}$



This gives a **FAST** method to generate projectors in the symplectic picture

To project onto the $X$ and $Y$ basis, the same process as above will work except one needs to replace the $Z_{i}$ operators with $X_{i}$ or $Y_{i}$



In general for a bitstring $| b^{'} \rangle$, where the $i$-th bit is $\in \{0,1,+,-, +i, -i \}$

$$|b' \rangle \langle b'| = \frac{1}{2^{n}} \sum_{k \in  \mathcal{W}}\Bigg[ (-1)^{(b''\cdot k) \: mod \: 2}\bigg( \bigotimes_{j=0}^{n-1} \sigma_{j}^{k[j]}  \bigg) \Bigg]$$

where:
- $k[j]$ is the $j$-th bit of the bitstring $| k \rangle$ this determines each $\sigma \in \{I, X, Y, Z \}$:
    - $\sigma_{j}^{0} = I_{i}$
    - $\sigma_{j}^{1} = Z_{i}$
    - $\sigma_{j}^{-} = X_{i}$
    - $\sigma_{j}^{+} = X_{i}$
    - $\sigma_{j}^{-i} = Y_{i}$
    - $\sigma_{j}^{+i} = Y_{i}$
- and $b''$ is the mapping of $b'$ to a $\{0,1\}^{\otimes n}$ state
    - $- \mapsto 1$
    - $+ \mapsto 0$
    - $-i \mapsto 1$
    - $+i \mapsto 0$

This is what `get_PauliwordOp_projector` does

In [44]:
from symmer.symplectic import get_PauliwordOp_projector
from tqdm import tqdm_notebook

# projector onto |0 + > ⊗ I ⊗ |1 >
Proj = get_PauliwordOp_projector('0+I1') # <--- can use 0,1, +,-, *, % for Z, X, Y basis and I to leave qubit unchanged
Proj

 0.125+0.000j IIII +
-0.125+0.000j IIIZ +
 0.125+0.000j IXII +
-0.125+0.000j IXIZ +
 0.125+0.000j ZIII +
-0.125+0.000j ZIIZ +
 0.125+0.000j ZXII +
-0.125+0.000j ZXIZ

# FAST(er) decomposition of sparse matrix into Pauli operators

The projector $|i \rangle \langle j |$ defines an all zero matrix with a single one in the $i$-th row and $j$-th column.


Therefore a sparse matrix can be decomposed in an efficent manner (better than the $4^{n}$ trace approach that is)

Given the projector onto the zero state: 

$$|00...0 \rangle \langle 00...0| = \frac{1}{2^{n}} \sum_{k \in  \mathcal{W}}\Bigg[ \bigg( \bigotimes_{i=0}^{n-1} \sigma_{i}^{k[i]}  \bigg) \Bigg]$$

$$=  \frac{1}{2^{n}} \Big(II\text{...}I + II\text{...}Z +\text{...} + ZZ\text{...}Z \Big) $$


- note this has $2^{n}$ terms!

The **Heisenberg picture** lets us convert it into an arbitrary projector of $|i \rangle \langle j |$, but multiplying the appropriate operators on the left and right.

Each multiplication takes $2^{n}$ (of which there are two)

What is the cost of decomposing an arbitrary matrix $M$ using this strategy?

Given the all zero projector.
Find all non-zero row and column pairs $i,j$ in the matrix $M$.

1. Build the projector: $|i \rangle \langle j |$ 
2. multiply the new projector by $M[i,j]$
3. add to overall decomposition



How does this scale?
- step 1. requires two multiplications of a single operator over a $2^{n}$ size term
- step 2. is a multiplication of  $2^{n}$ coefficients by a single number
- step 3. addition to tracking term (at worst the size of this object will become $4^{n})$


What is the computation cost?

$$\mathcal{O}(S_{M} \cdot 2^{n})$$

- for each element in $M$ (total denoted $S_{M}$)
- build projector and multiply by $M[i,j]$


WHEREAS standard method does:
1. Build each $4^{n}$ pauli matrices
2. Multiply the matrix $M$ by each of the matrix for of each Pauli operators
3. take trace
4. each operation gives coeff weight

A dense verion would require:
$$4^{n} \times \underbrace{\big[ (2^{n})^{2} \big]}_{\text{Frobenius inner product}} = 2^{4n}$$

Whereas a  sparse form would need:

$$4^{n} \times \underbrace{\big[ min(S_{M}, S_{P}) \big]^{2}}_{\text{Frobenius inner product (sparse)}}$$
where $S_{P} = (2^{n})$. Note we have ignored the final sum taken once the elementwise multication is done. Therefore, to obtain an advantage need $S_{M}< S_{P}$:

$$= 4^{n} S_{M}^{2}$$

The new approach improves upon this by:

$$\frac{(4^{n} \cdot S_{M}^{2})}{(S_{M} \cdot 2^{n})} = 2^{n} \cdot S_{M}$$
- note at maximum sparcity $S_{M}^{2} = S_{M} = 2^{2n}$ (as cannot have more nonzero elements than the size of the matrix)

which is an **exponential speedup!**

In the dense version:

$$\frac{2^{4n}}{(S_{M} \cdot 2^{n})} = \frac{1}{S_{M}} (2^{3n})$$

with a completely dense matrix $S_{M} = 2^{2n}$:

$$\frac{(2^{4n})}{(2^{2n} \cdot 2^{n})} = \frac{(2^{4n})}{(2^{3n})} = 2^{n}$$

# Matrix to Pauli Code

In [45]:
from symmer.symplectic import PauliwordOp
import numpy as np
from scipy.sparse import rand

In [46]:
n_qubits = 7

D = 2**n_qubits
x = rand(D, D, density=0.1, format='csr')
x.shape

(128, 128)

In [38]:
# n_qubits = 12

# D = 2**n_qubits
# x = rand(D, D, density=0.00001, format='csr')
# x.shape

In [47]:
mat = x.toarray()
mat.nonzero()[0].shape

(1638,)

In [48]:
Pop = PauliwordOp.from_matrix(mat, strategy='projector')
Pop.n_terms

Building operator via projectors:   0%|          | 0/1638 [00:00<?, ?it/s]

16384

In [49]:
# dense

Pop2 = PauliwordOp.from_matrix(mat, strategy='full_basis')
Pop2.n_terms

Building operator via full basis:   0%|          | 0/16384 [00:00<?, ?it/s]

16384

In [42]:
# sparse 
Pop_sparse = PauliwordOp.from_matrix(x, strategy='full_basis')
Pop_sparse.n_terms

Building operator via full basis:   0%|          | 0/16384 [00:00<?, ?it/s]

16384

In [43]:
# check decomposition
np.allclose(Pop.to_sparse_matrix.toarray(),
            mat)

True

In [19]:
# expensive for dense scaling:
2**(4*n_qubits)

16777216

In [20]:
# vs projector
2**(n_qubits) * len(mat.nonzero()[0])

2624

In [21]:
# strategy == projector or full_basis
%timeit Pop = PauliwordOp.from_matrix(mat, strategy='projector')

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

...

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

Building operator via projectors:   0%|          | 0/41 [00:00<?, ?it/s]

53.6 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%timeit Pop2 = PauliwordOp.from_matrix(mat, strategy='full_basis')

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

Building operator via full basis:   0%|          | 0/4096 [00:00<?, ?it/s]

3.22 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
